In [1]:
import pandas as pd
import json
from math import cos, asin, sqrt

Lat/Lng data copied from https://www.latlong.net/category/national-parks-236-42.html into a spreadsheet and saved out as a CSV file.

In [2]:
park_coords = pd.read_csv("./Resources/parks_lat_lng.csv")
park_coords

,Place Name,Latitude,Longitude
0,"Jackson Hole, WY, USA",43.582767,-110.821999
1,"Mojave National Preserve, Kelso, CA, USA",35.141689,-115.510399
2,"Joshua Tree National Park, California, USA",33.881866,-115.900650
3,"Buffalo National River, St Joe, AR, USA",35.985512,-92.757652
4,"Hot Springs National Park, Hot Springs, AR, USA",34.521530,-93.042267
...,...,...,...
80,"Mount Mitchell State Park, NC, the US",35.768803,-82.306137
81,"Mt. Hood National Forest, Oregon, the US",45.454350,-121.933136
82,"Island Park, Idaho, the US",44.429733,-111.440849
83,"Redwood National Park, CA, USA",41.213181,-124.004631


In [3]:
names = []
locations = []
for _, place in park_coords.iterrows():
    names.append(place['Place Name'].split(',',1)[0])
names

['Jackson Hole',
 'Mojave National Preserve',
 'Joshua Tree National Park',
 'Buffalo National River',
 'Hot Springs National Park',
 'Kartchner Caverns State Park',
 'Navajo Nation Reservation',
 'Sipsey Wilderness',
 'Wrangell-St. Elias National Park & Preserve',
 'Lake Clark National Park and Preserve',
 'Katmai National Park and Preserve',
 'Glacier Bay National Park and Preserve',
 'Turtle Mountain',
 'Odiorne Point State Park',
 'Channel Islands National Park',
 'Maplewood State Park',
 'Bear Butte State Park',
 'Okefenokee National Wildlife Refuge',
 'Denali National Park and Preserve',
 'Acadia National Park',
 'Topanga State Park',
 'Gates of the Arctic National Park and Preserve',
 'Kaloko-Honok?hau National Historical Park',
 'Pedernales Falls State Park',
 'Whiskeytown National Recreation Area',
 'Mesa Verde National Park',
 'Arches National Park',
 'Mount Rainier National Park',
 'Kenai Fjords National Park',
 'Anahuac National Wildlife Refuge',
 'Bandelier National Monume

In [29]:
park_details = pd.DataFrame(data=names, columns=['name'])
park_details['latitude'] = park_coords['Latitude']
park_details['longitude'] = park_coords['Longitude']
park_details

,name,latitude,longitude
0,Jackson Hole,43.582767,-110.821999
1,Mojave National Preserve,35.141689,-115.510399
2,Joshua Tree National Park,33.881866,-115.900650
3,Buffalo National River,35.985512,-92.757652
4,Hot Springs National Park,34.521530,-93.042267
...,...,...,...
80,Mount Mitchell State Park,35.768803,-82.306137
81,Mt. Hood National Forest,45.454350,-121.933136
82,Island Park,44.429733,-111.440849
83,Redwood National Park,41.213181,-124.004631


Weather station data taken from https://www1.ncdc.noaa.gov/pub/data/inventories/MASTER-STN-HIST.TXT (also available as a 3.9Mb zip: https://www1.ncdc.noaa.gov/pub/data/inventories/MASTER-STN-HIST.ZIP).

File was:
* downloaded
* manually processed to remove extraneous columns through the use of Notepad++ column editing,
* a macro was used to remove entries with no WBAN number
* double spaces were converted to commas
* single spaces were converted to commas
* the file saved out again as a CSV
* CSV file imported into Excel
* sorted by WBAN, then by END_DATE (desc)
* lat and lng entries converted to decimal
* time gaps calculated
* changes in position were also calculated, but for our purposes were determined to be insignificant and left out
* file saved out again

In [5]:
stations_hist_df = pd.read_csv("./Resources/wban_station_hist.csv")
stations_hist_df

,wban,start_date,end_date,lat_deg,lat_min,lat_sec,lng_deg,lng_min,lng_sec,dec_lat,dec_lng,time_gap,gap_calc
0,100,19600801,99991231,34,5,59,-93,3,57,34.099722,-93.065833,0,0
1,101,10101,99991231,43,4,1,74,28,59,43.066944,75.450000,0,0
2,102,19580701,99991231,66,58,59,-160,25,59,66.983056,-160.433056,0,0
3,103,19541001,99991231,65,37,1,-168,6,0,65.616944,-168.100000,0,0
4,104,10101,99991231,63,1,1,-154,22,1,63.016944,-154.366944,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
23802,96407,20150814,20160129,66,33,43,-159,0,13,66.561944,-159.003611,0,0
23803,96407,20160129,99991231,66,33,43,-159,0,13,66.561944,-159.003611,0,0
23804,96408,20150820,20160129,63,27,7,-150,52,29,63.451944,-150.874722,0,0
23805,96408,20160129,99991231,63,27,7,-150,52,29,63.451944,-150.874722,0,0


### Consolidate rows so we only have 1 per unique wban number.

In [6]:
uniq_wbans = stations_hist_df['wban'].unique()
len(uniq_wbans)

6871

### We'll be keeping rows with the highest end_date, so get the oldest start_dates to use with them.

In [7]:
starts = stations_hist_df.groupby('wban')['start_date'].min()
starts

wban
100      19600801
101         10101
102      19580701
103      19541001
104         10101
           ...   
96405    20170729
96406    20140828
96407    20150814
96408    20150820
96409    20170822
Name: start_date, Length: 6871, dtype: int64

In [8]:
# reduce df to 1 row per station: the newest
stations_df = stations_hist_df.drop_duplicates(subset='wban', keep='last')
stations_df

,wban,start_date,end_date,lat_deg,lat_min,lat_sec,lng_deg,lng_min,lng_sec,dec_lat,dec_lng,time_gap,gap_calc
0,100,19600801,99991231,34,5,59,-93,3,57,34.099722,-93.065833,0,0
1,101,10101,99991231,43,4,1,74,28,59,43.066944,75.450000,0,0
2,102,19580701,99991231,66,58,59,-160,25,59,66.983056,-160.433056,0,0
3,103,19541001,99991231,65,37,1,-168,6,0,65.616944,-168.100000,0,0
4,104,10101,99991231,63,1,1,-154,22,1,63.016944,-154.366944,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
23798,96405,20170729,99991231,60,28,23,-145,21,15,60.473056,-145.354167,0,0
23801,96406,20160129,99991231,64,30,5,-154,7,47,64.501389,-154.129722,0,0
23803,96407,20160129,99991231,66,33,43,-159,0,13,66.561944,-159.003611,0,0
23805,96408,20160129,99991231,63,27,7,-150,52,29,63.451944,-150.874722,0,0


In [9]:
int(stations_df.loc[23805]['start_date'])

20160129

In [10]:
# replace start_date with oldest start_date
stations_df = stations_df.assign(start_date=starts.values)

In [11]:
int(stations_df.loc[23805]['start_date'])

20150820

### We also want to throw out anything that has a significant time gap (i.e., >= 20000 is guaranteed to be more than 1 year).

In [12]:
stations_df['time_gap'].max()

699689

In [13]:
stations_df.loc[stations_df['time_gap']>=20000]

,wban,start_date,end_date,lat_deg,lat_min,lat_sec,lng_deg,lng_min,lng_sec,dec_lat,dec_lng,time_gap,gap_calc
450,3016,19690911,99991231,39,31,40,-107,43,11,39.527778,-107.719722,119777,0
462,3024,19510501,99991231,35,41,42,-101,23,42,35.695000,-101.395000,250588,250588
694,3122,19530501,99991231,33,48,6,-118,20,31,33.801667,-118.341944,79770,0
761,3145,19600701,99991231,32,39,0,-114,37,0,32.650000,-114.616667,79270,0
888,3181,19730201,99991231,37,38,0,-118,51,0,37.633333,-118.850000,39670,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
23725,94976,19721101,99991231,44,27,0,-95,49,0,44.450000,-95.816667,30870,0
23734,94979,19520501,99991231,41,50,0,-90,20,0,41.833333,-90.333333,29621,0
23742,94982,19500401,99991231,41,36,42,-90,34,51,41.611667,-90.580833,100470,-80040324
23766,94988,19680801,99991231,42,6,38,-92,54,58,42.110556,-92.916111,20100,0


In [14]:
rows = stations_df.loc[stations_df['time_gap']>=20000].index
rows

Int64Index([  450,   462,   694,   761,   888,   928,  1066,  1096,  1142,
             1160,
            ...
            23556, 23593, 23655, 23672, 23685, 23725, 23734, 23742, 23766,
            23772],
           dtype='int64', length=486)

In [15]:
stations_df = stations_df.drop(labels=rows)
stations_df

,wban,start_date,end_date,lat_deg,lat_min,lat_sec,lng_deg,lng_min,lng_sec,dec_lat,dec_lng,time_gap,gap_calc
0,100,19600801,99991231,34,5,59,-93,3,57,34.099722,-93.065833,0,0
1,101,10101,99991231,43,4,1,74,28,59,43.066944,75.450000,0,0
2,102,19580701,99991231,66,58,59,-160,25,59,66.983056,-160.433056,0,0
3,103,19541001,99991231,65,37,1,-168,6,0,65.616944,-168.100000,0,0
4,104,10101,99991231,63,1,1,-154,22,1,63.016944,-154.366944,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
23798,96405,20170729,99991231,60,28,23,-145,21,15,60.473056,-145.354167,0,0
23801,96406,20140828,99991231,64,30,5,-154,7,47,64.501389,-154.129722,0,0
23803,96407,20150814,99991231,66,33,43,-159,0,13,66.561944,-159.003611,0,0
23805,96408,20150820,99991231,63,27,7,-150,52,29,63.451944,-150.874722,0,0


In [16]:
stations_df['time_gap'].max()

19971

#### After completing most of the above work, I discovered the HOMR list of WBAN numbers, which would have been just as difficult to parse.  https://www.ncdc.noaa.gov/homr/reports/platforms

### Find Closest Weather Stations to Each Park 
Algorithm was adapated from this answer on StackOverflow:
https://stackoverflow.com/questions/41336756/find-the-closest-latitude-and-longitude

In [17]:
def distance(lat1, lng1, lat2, lng2):
    p = 0.017453292519943295
    a = 0.5 - cos((lat2-lat1)*p)/2 + cos(lat1*p)*cos(lat2*p) * (1-cos((lng2-lng1)*p)) / 2
    return 12742 * asin(sqrt(a))

def closest(coords, park):
    return min(coords, key=lambda stn: distance(park['lat'], park['lng'], stn['lat'], stn['lng']))

### UNFORTUNATELY....
The BigQuery gsod sample is obviously incomplete (it's only a sample), and its data is suspect. First off, there are only results for 2540 WBAN station numbers, whereas we identified 6871 unique stations. Secondly, of those 2540 stations in the sample data, 23 of them representing 62030 rows of data, have invalid WBAN numbers (i.e., numbers below 100).

#### Limit Our List of Stations to Only Those That Are Valid, and Known to Have Data in the Sample

In [18]:
sample_stations = pd.read_csv("./Resources/sample_data_stations.csv")['wban'].values
sample_stations

array([    1,     2,     3, ..., 94999, 96401, 96402])

In [19]:
coords = []
for _, row in stations_df.iterrows():
    if row['wban'] in sample_stations:
        coords.append({'wban': int(row['wban']), 'lat': row['dec_lat'], 'lng': row['dec_lng']})
coords

[{'wban': 107, 'lat': 59.45, 'lng': -157.3330555556},
 {'wban': 108, 'lat': 61.85, 'lng': -165.5669444444},
 {'wban': 109, 'lat': 58.983055555600004, 'lng': -159.05},
 {'wban': 114, 'lat': 48.6, 'lng': -113.1169444444},
 {'wban': 137, 'lat': 36.4219444444, 'lng': -105.29},
 {'wban': 142, 'lat': 37.45, 'lng': -94.73305555559999},
 {'wban': 152, 'lat': 44.933055555600006, 'lng': -73.1},
 {'wban': 155, 'lat': 37.85, 'lng': -76.88305555560001},
 {'wban': 166, 'lat': 30.5, 'lng': -97.9669444444},
 {'wban': 175, 'lat': 32.15, 'lng': -94.85},
 {'wban': 178, 'lat': 40.6169444444, 'lng': -74.25},
 {'wban': 191, 'lat': 33.1830555556, 'lng': -80.0330555556},
 {'wban': 193, 'lat': 33.65, 'lng': -81.6830555556},
 {'wban': 209, 'lat': 33.6, 'lng': -97.7830555556},
 {'wban': 222, 'lat': 38.35, 'lng': -93.68305555559999},
 {'wban': 230, 'lat': 30.395, 'lng': -97.56694444440001},
 {'wban': 231, 'lat': 36.33, 'lng': -77.635},
 {'wban': 255, 'lat': 29.4830555556, 'lng': -93.6330555556},
 {'wban': 265, 'l

In [20]:
# tempDataList = [{'lat': 39.7612992, 'lon': -86.1519681}, 
#                 {'lat': 39.762241,  'lon': -86.158436 }, 
#                 {'lat': 39.7622292, 'lon': -86.1578917}]

# v = {'lat': 39.7622290, 'lon': -86.1519750}
# print(closest(tempDataList, v))
closests =[]
stations = {}
for _, park in park_coords.iterrows():
    coord = {'lat': park['Latitude'], 'lng': park['Longitude']}
    c = closest(coords, coord)
    wban = c['wban']
    closests.append(wban)
    if not wban in stations:
        stations[wban] = {'lat': c['lat'], 'lng': c['lng']}
closests

[24166,
 23192,
 23172,
 53918,
 3962,
 53146,
 3029,
 63895,
 46404,
 26546,
 25521,
 25316,
 94928,
 4743,
 53152,
 94966,
 24006,
 63890,
 96401,
 14616,
 93197,
 26542,
 21510,
 166,
 4222,
 3061,
 93075,
 24237,
 26438,
 12923,
 93091,
 3872,
 94173,
 54739,
 63844,
 93198,
 53150,
 94173,
 22525,
 93197,
 53149,
 93032,
 92827,
 93065,
 3077,
 94041,
 63811,
 53978,
 4236,
 23157,
 13876,
 94173,
 53870,
 22521,
 92809,
 94866,
 54912,
 24152,
 24230,
 3065,
 3065,
 13891,
 3802,
 12871,
 12841,
 63885,
 14894,
 63813,
 13731,
 14777,
 3974,
 22526,
 3858,
 21510,
 24153,
 3040,
 94823,
 12841,
 13983,
 12853,
 53877,
 24238,
 94163,
 24283,
 94173]

In [21]:
len(closests)

85

In [22]:
with open('./Results/stations.json', 'w', encoding='utf-8') as f:
    json.dump(stations, f, ensure_ascii=False, indent=4)

In [23]:
len(stations)

78

In [30]:
park_details['wban'] = closests
park_details

,name,latitude,longitude,wban
0,Jackson Hole,43.582767,-110.821999,24166
1,Mojave National Preserve,35.141689,-115.510399,23192
2,Joshua Tree National Park,33.881866,-115.900650,23172
3,Buffalo National River,35.985512,-92.757652,53918
4,Hot Springs National Park,34.521530,-93.042267,3962
...,...,...,...,...
80,Mount Mitchell State Park,35.768803,-82.306137,53877
81,Mt. Hood National Forest,45.454350,-121.933136,24238
82,Island Park,44.429733,-111.440849,94163
83,Redwood National Park,41.213181,-124.004631,24283


In [31]:
park_details.to_json(path_or_buf="./Results/parks.json")

Now we run the following query on the Google Cloud Platform's BigQuery gsod sample dataset, which can be found here:
https://console.cloud.google.com/bigquery?p=bigquery-public-data&d=samples&t=gsod&page=table&_ga=2.193870957.-1150284291.1573724531

```
SELECT wban_number, month, 
    AVG(mean_temp) as avg_temp, 
    AVG(mean_visibility) as avg_visability, 
    AVG(mean_wind_speed) as avg_wind_speed, 
    AVG(total_precipitation) as avg_precipitation, 
    AVG(snow_depth) as avg_snow_depth
FROM `bigquery-public-data.samples.gsod` 
WHERE wban_number = 24166
    OR wban_number = 23192
    OR wban_number = 23172
    OR wban_number = 53918
    OR wban_number = 3962
    OR wban_number = 53146
    OR wban_number = 3029
    OR wban_number = 63895
    OR wban_number = 46404
    OR wban_number = 26546
    OR wban_number = 25521
    OR wban_number = 25316
    OR wban_number = 94928
    OR wban_number = 4743
    OR wban_number = 53152
    OR wban_number = 94966
    OR wban_number = 24006
    OR wban_number = 63890
    OR wban_number = 96401
    OR wban_number = 14616
    OR wban_number = 93197
    OR wban_number = 26542
    OR wban_number = 21510
    OR wban_number = 166
    OR wban_number = 4222
    OR wban_number = 3061
    OR wban_number = 93075
    OR wban_number = 24237
    OR wban_number = 26438
    OR wban_number = 12923
    OR wban_number = 93091
    OR wban_number = 3872
    OR wban_number = 94173
    OR wban_number = 54739
    OR wban_number = 63844
    OR wban_number = 93198
    OR wban_number = 53150
    OR wban_number = 94173
    OR wban_number = 22525
    OR wban_number = 93197
    OR wban_number = 53149
    OR wban_number = 93032
    OR wban_number = 92827
    OR wban_number = 93065
    OR wban_number = 3077
    OR wban_number = 94041
    OR wban_number = 63811
    OR wban_number = 53978
    OR wban_number = 4236
    OR wban_number = 23157
    OR wban_number = 13876
    OR wban_number = 94173
    OR wban_number = 53870
    OR wban_number = 22521
    OR wban_number = 92809
    OR wban_number = 94866
    OR wban_number = 54912
    OR wban_number = 24152
    OR wban_number = 24230
    OR wban_number = 3065
    OR wban_number = 3065
    OR wban_number = 13891
    OR wban_number = 3802
    OR wban_number = 12871
    OR wban_number = 12841
    OR wban_number = 63885
    OR wban_number = 14894
    OR wban_number = 63813
    OR wban_number = 13731
    OR wban_number = 14777
    OR wban_number = 3974
    OR wban_number = 22526
    OR wban_number = 3858
    OR wban_number = 21510
    OR wban_number = 24153
    OR wban_number = 3040
    OR wban_number = 94823
    OR wban_number = 12841
    OR wban_number = 13983
    OR wban_number = 12853
    OR wban_number = 53877
    OR wban_number = 24238
    OR wban_number = 94163
    OR wban_number = 24283
    OR wban_number = 94173
GROUP BY wban_number, month
ORDER BY wban_number, month;
```

#### Here's the output:

In [26]:
results_df = pd.read_json("./Resources/results.json")
results_df

,wban_number,month,avg_temp,avg_visability,avg_wind_speed,avg_precipitation,avg_snow_depth
0,166,1,79.015982,NaN,6.690639,0.000000,NaN
1,166,2,78.730843,NaN,6.536627,0.000000,NaN
2,166,3,76.754608,NaN,6.790553,0.000000,NaN
3,166,4,72.855000,NaN,6.410000,0.000000,NaN
4,166,5,66.831336,NaN,5.740092,0.000000,NaN
...,...,...,...,...,...,...,...
801,96401,8,52.817241,23.234483,4.521429,0.189600,NaN
802,96401,9,45.682758,24.848276,4.472414,0.083077,NaN
803,96401,10,35.756667,24.546667,4.723333,0.085200,NaN
804,96401,11,16.477778,15.988889,7.744444,0.079259,NaN


In [27]:
results_df.fillna(value = 0, inplace = True)
results_df

,wban_number,month,avg_temp,avg_visability,avg_wind_speed,avg_precipitation,avg_snow_depth
0,166,1,79.015982,0.000000,6.690639,0.000000,0.0
1,166,2,78.730843,0.000000,6.536627,0.000000,0.0
2,166,3,76.754608,0.000000,6.790553,0.000000,0.0
3,166,4,72.855000,0.000000,6.410000,0.000000,0.0
4,166,5,66.831336,0.000000,5.740092,0.000000,0.0
...,...,...,...,...,...,...,...
801,96401,8,52.817241,23.234483,4.521429,0.189600,0.0
802,96401,9,45.682758,24.848276,4.472414,0.083077,0.0
803,96401,10,35.756667,24.546667,4.723333,0.085200,0.0
804,96401,11,16.477778,15.988889,7.744444,0.079259,0.0


In [28]:
results_df.to_json(path_or_buf="./Results/weather.json")